In [1]:
# Import Libraries

from collections import defaultdict
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from Bio import Entrez, Medline
from pathlib import Path
import pubchempy as pcp
import networkx as nx
from tqdm import tqdm
import pandas as pd
import numpy as np
import requests
import signal
import time
import re

# Set email address (required for NCBI API usage)
Entrez.email = 'sks6765@psu.edu'

In [ ]:
def create_graph_viz(dataframe, column1, column2, label=1):
    # Creating an interactive graph
    G = nx.Graph()
    
    # Add edges to the graph
    for index, row in dataframe.iterrows():
        G.add_edge(row[column1], row[column2], label=row[label] if label != 1 else 1)
    
    # Add edge labels
    edge_labels = {(u, v): d['label'] for u, v, d in G.edges(data=True)}
    nx.set_edge_attributes(G, edge_labels, 'label')
    
    # Position nodes using spring layout
    pos = nx.spring_layout(G)
    
    # Extract node and edge information
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    edge_x = []
    edge_y = []
    edge_text = []
    for edge in G.edges():
        source, target = edge
        x0, y0 = pos[source]
        x1, y1 = pos[target]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_text.append(G.get_edge_data(*edge)['label'])  # Use existing edge labels
    
    # Create a Plotly figure
    fig = go.Figure(data=[go.Scatter(
        x=edge_x,
        y=edge_y,
        mode='lines',
        line_shape='spline',
        opacity=0.5,
        hoverinfo='none'
    ),
    go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers',
        hoverinfo='text',
        hovertext=[f'Node {node}' for node in G.nodes()]
    ),
    go.Scatter(
        x=[(pos[edge[0]][0] + pos[edge[1]][0]) / 2 for edge in G.edges()],
        y=[(pos[edge[0]][1] + pos[edge[1]][1]) / 2 for edge in G.edges()],
        mode='text',
        text=edge_text,
        textposition='middle center',
        hoverinfo='none'
    )])
    
    # Customize the layout
    fig.update_layout(
        showlegend=False,
        hovermode='x',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    
    # Display the figure
    fig.show()

#### Extra code - might use later. This is to retrieve info from the gene db.

In [ ]:
"""
NOTES/OBSERVATIONS:

1. I decided to do a_is_acting == 't' as otherwise, we would get matches like this which, imo, doesn't really make sense, i.e., 
what does a protein "acting" on a drug mean? - drug acting on a protein makes more sense.

8063 	9606.ENSP00000320025 	CIDs00003333 	inhibition 	inhibition 	f 	991
8064 	CIDs00003333 	9606.ENSP00000320025 	inhibition 	inhibition 	t 	991

2. In theory, actions.query("(item_id_a in @l1) and (item_id_b in @l2)") should return 100 rows for each interaction. However, there are 
certain dpi's like below that have different interaction scores, whether a acts on b, etc. Thus, we filter out those with our conditions.

21666299 	CIDs00060795 	9606.ENSP00000289753 	inhibition 	inhibition 	t 	958
21666301 	CIDs00060795 	9606.ENSP00000289753 	binding 	NaN 	t 	800

3. There are certain pairs like, ('CIDs00002435', '9606.ENSP00000398832') that do not have an entry in the database either as
(item_id_a, item_id_b) or (item_id_b, item_id_a) - This in turn reduces the number of samples.

4. I am bound to see more a_is_acting = f since it is the major element in the actions db by about 30 times more than t.
a_is_acting
f    21078378
t      695113.

5. Even if I consider a->b and b->a relationships, I'm getting certain useless pairs such as the following which doesn't really give me any
additional information.
1448204 	9606.ENSP00000241256 	CIDs00002477 	binding 	NaN 	f 	894
1448205 	CIDs00002477 	9606.ENSP00000241256 	binding 	NaN 	f 	894

6. This will return a lower number than the rows in r since there can be multiple relationships between the same drug-protein pair.
r = actions.query("item_id_a in @l1")
r = r[r.apply(lambda x: (x["item_id_a"], x["item_id_b"]) in target_pairs, axis=1)]
len(set([(x.item_id_a, x.item_id_b) for x in r.itertuples()]))

7. Out of 200 possible (a,b) and (b,a) combinations, 154 exist in the database.
r = actions.query("item_id_a in @l1 or item_id_b in @l1")
len(set(target_pairs).intersection(set([(x.item_id_a, x.item_id_b) for x in r.itertuples()]).union(set([(x.item_id_b, x.item_id_a) for x in r.itertuples()]))))
"""
# Previous code - not needed now.
l1 = [x[0] for x in target_pairs]
'''
I'm only searching one column (item_id_a) with the CID's. All of my CID's WILL be present in that column, since each CID is mentioned once
in item_id_a and another in item_id_b.
'''
r = actions.query("item_id_a in @l1")

In [ ]:
def get_protein_info(record):
    return f"{record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']} ({record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_desc']})",\
    record[0]["Entrezgene_summary"],\
    record[0]["Entrezgene_prot"]

cleaned_gene_id = int(re.sub("9606.ENSP0*", "", gene))
Entrez.email = "sample_email@example.org" #Doesn't really matter
handle = Entrez.efetch(db="gene", id=cleaned_gene_id, rettype="gb", retmode="xml")
record = Entrez.read(handle)

try:
    all_information["gene name"], all_information["gene summary"], all_information["proteins from gene"] = get_protein_info(record)
except:
    continue

Entrez.email = "sample_email@example.org" #Doesn't really matter
handle = Entrez.efetch(db="gene", id=7448, rettype="gb", retmode="xml")
record = Entrez.read(handle)

all_information = {}
all_information["gene name"] = f"{record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']} ({record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_desc']})"
all_information["gene summary"] = record[0]["Entrezgene_summary"]
all_information["proteins from gene"] = record[0]["Entrezgene_prot"]
all_information

## DDI sampling

In [ ]:
ddi = pd.read_csv("bio-decagon-combo.csv")

In [ ]:
class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError

def CID_information(CID: int, timeout: int = 5):

    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)

    # Set the alarm
    signal.alarm(timeout)

    try:
        cleaned_cid = int(re.sub(r"CID[m|s]*0*", "", CID))
        compound = pcp.Compound.from_cid(cleaned_cid)
        try:
            compound_name = compound.synonyms[0]
        except:
            compound_name = compound.iupac_name
        return (compound_name, compound.canonical_smiles)
    except TimeoutError:
        print("Timeout: Function took too long to complete.")
        return None
    finally:
        # Disable the alarm
        signal.alarm(0)

In [ ]:
number_of_ddis_to_sample = 10000
number_of_ddis_to_retain = 300
all_information = []
sampled_ddis = ddi.sample(n = number_of_ddis_to_sample, random_state = 1)

In [ ]:
def save_dataframe(all_information):
    pd.DataFrame(all_information, columns=["drug_1_CID", "drug_1_name", "drug_1_SMILES", "relationship", "drug_2_CID", "drug_2_name", 
                                               "drug_2_SMILES"]).to_csv("DDI_subset.csv", index=False)

for idx, sample in tqdm(enumerate(sampled_ddis.itertuples(index=False))):    
    drug_1_CID = sample[0]
    drug_2_CID = sample[1]
    relationship = sample[3]

    drug_1_info = CID_information(drug_1_CID)
    drug_2_info = CID_information(drug_2_CID)
    
    try:
        # I want the drug name to not include any numbers or brackets, i.e., only keep regular words.
        if (not re.search(r"[\W\d]|(sulfate)", drug_1_info[0])) and (not re.search(r"[\W\d]|(sulfate)", drug_2_info[0])):

            # This ensures that when I am retrieving the background information, I will get 20 pubmed hits.
            drug1_query = f"{drug_1_info[0]} AND hasabstract AND Humans AND (AD OR AE OR PK OR PD OR CO OR TU OR DE)"
            with Entrez.esearch(db='pubmed', term=drug1_query, retmax=20, sort="relevance") as handle:
                paper_list_drug1 = Entrez.read(handle)["IdList"]
    
            drug2_query = f"{drug_2_info[0]} AND hasabstract AND Humans AND (AD OR AE OR PK OR PD OR CO OR TU OR DE)"
            with Entrez.esearch(db='pubmed', term=drug2_query, retmax=20, sort="relevance") as handle:
                paper_list_drug2 = Entrez.read(handle)["IdList"]
            
            if len(paper_list_drug1) == 20 and len(paper_list_drug2) == 20:            
                all_information.append((drug_1_CID, drug_1_info[0], drug_1_info[1], relationship, drug_2_CID, drug_2_info[0], drug_2_info[1]))
    except:
        continue

    # timeout after 20 API calls.
    if (idx % 20 == 0) and (idx != 0):
        print(f"Samples collected so far: {len(all_information)}. Saving checkpoint and cooling down ...")
        save_dataframe(all_information)
        time.sleep(60)        
    
    if len(all_information) == number_of_ddis_to_retain:
        print("Necessary number of DDIs obtained... exiting")
        save_dataframe(all_information)
        break

## DPI sampling

In [ ]:
# Processing actions dataframe

actions = pd.read_csv("../base_data/9606.actions.v5.0.tsv", sep="\t") #drug-protein pairs with labels
actions = actions[actions['item_id_a'].str.startswith("CIDs")]
actions = actions.query("score > 900 and a_is_acting == 't'").dropna() # Only retain "strong" interactions
actions["item_id_a"] = actions["item_id_a"].str.replace("CIDs", "CID0")
actions.to_csv("action_subset.csv", index=False)

In [ ]:
ddi_subset = pd.read_csv("DDI_subset.csv")
dpi = pd.read_csv("bio-decagon-targets-all.csv")
actions_subset = pd.read_csv("action_subset.csv")

all_drugs_CID = set(ddi_subset["drug_1_CID"].to_list() + ddi_subset["drug_2_CID"].to_list())

# The query returns A LOT of DPIs. We take only a subset of them.
dpi_subset = dpi.query("STITCH in @all_drugs_CID").sample(n=5000, random_state=1)

# Converting PubChem CID to STITCH CIDs (singular element) (https://www.biostars.org/p/155342/)
dpi_subset['STITCH'] = dpi_subset['STITCH'].str.replace('CID0', 'CIDs')

In [ ]:
search_list = []
for row in tqdm(dpi_subset.itertuples()):  
    # Retrieving protein data from the given gene ID
    url = f"https://string-db.org/api/json/get_string_ids?identifiers={row.Gene}&species=9606"
    try:
        response = requests.get(url).json()[0]
        string_id = (response["stringId"], response["preferredName"], response["annotation"], row.Gene)
    except:
        continue

    search_list.append((row.STITCH, string_id))

target_pairs = list(zip([x[0] for x in search_list], [x[1][0] for x in search_list]))
'''
1. I can do the following check since if (a, b) does not exist in the db, (b, a) also wont.
'''
r = actions_subset[actions_subset.apply(lambda x: (x["item_id_a"], x["item_id_b"]) in set(target_pairs), axis=1)]

search_df = pd.DataFrame({"cid": [x[0] for x in search_list], "protein_number": [x[1][0] for x in search_list], 
                          "protein_name": [x[1][1] for x in search_list], "protein_desc": [x[1][2] for x in search_list],
                          "gene": [x[1][3] for x in search_list]})

r = pd.merge(r, search_df, left_on=["item_id_a", "item_id_b"], right_on=["cid", "protein_number"], 
             how="inner").drop(columns=["item_id_a", "item_id_b", "a_is_acting", "score"]).drop_duplicates() # Need to drop duplicates because of "a is acting" column. That is, the same entry can be repeated twice because of "a is acting"

r["cid"] = r["cid"].str.replace("CIDs", "CID0") # Going back to PubChem CID to be compatible with other DDI
r.to_csv("DPI_subset.csv", index=False)

In [ ]:
target_pairs = list(zip([x[0] for x in search_list], [x[1][0] for x in search_list]))
'''
1. I can do the following check since if (a, b) does not exist in the db, (b, a) also wont.
'''
r = actions_subset[actions_subset.apply(lambda x: (x["item_id_a"], x["item_id_b"]) in set(target_pairs), axis=1)]

search_df = pd.DataFrame({"cid": [x[0] for x in search_list], "protein_number": [x[1][0] for x in search_list], 
                          "protein_name": [x[1][1] for x in search_list], "protein_desc": [x[1][2] for x in search_list],
                          "gene": [x[1][3] for x in search_list]})

r = pd.merge(r, search_df, left_on=["item_id_a", "item_id_b"], right_on=["cid", "protein_number"], 
             how="inner").drop(columns=["item_id_a", "item_id_b", "a_is_acting", "score"]).drop_duplicates() # Need to drop duplicates because of "a is acting" column. That is, the same entry can be repeated twice because of "a is acting"

r["cid"] = r["cid"].str.replace("CIDs", "CID0") # Going back to PubChem CID to be compatible with other DDI
r.to_csv("DPI_subset.csv", index=False)

In [ ]:
# Adding the drug name to the DPI dataframe for easy access
ddi_subset = pd.read_csv("DDI_subset.csv")
dpi_subset = pd.read_csv("DPI_subset.csv")

d1 = pd.Series(ddi_subset.drug_1_name.values,index=ddi_subset.drug_1_CID).to_dict()
d2 = pd.Series(ddi_subset.drug_2_name.values,index=ddi_subset.drug_2_CID).to_dict()
all_drug_cid_name = {**d1, **d2}

all_drug_cid_name_df = pd.DataFrame(all_drug_cid_name.items(), columns=["cid", "drug_name"])
dpi_subset = pd.merge(dpi_subset, all_drug_cid_name_df, left_on=["cid"], right_on=["cid"])
dpi_subset.to_csv("DPI_subset.csv", index=False)

## PPI sampling

In [ ]:
ppi = pd.read_csv("bio-decagon-ppi.csv")
dpi_subset = pd.read_csv("DPI_subset.csv")

In [ ]:
all_genes = set(dpi_subset["gene"].to_list())

In [ ]:
# Using this now to have more number of proteins.
r = ppi.query("`Gene 1` in @all_genes or `Gene 2` in @all_genes").sample(n=500, random_state=1)

In [ ]:
# I use this condition since I don't want to introduce new genes as a part of the dataset. - not using this anymore - getting much less proteins
r = ppi[ppi["Gene 1"].isin(all_genes) & ppi["Gene 2"].isin(all_genes)]

In [ ]:
'''
nscore - neighborhood score, (computed from the inter-gene nucleotide count).
fscore - fusion score (derived from fused proteins in other species).
pscore - cooccurence score of the phyletic profile (derived from similar absence/presence patterns of genes).
ascore - coexpression score (derived from similar pattern of mRNA expression measured by DNA arrays and similar technologies).
escore - experimental score (derived from experimental data, such as, affinity chromatography).
dscore - database score (derived from curated data of various databases).
tscore - textmining score (derived from the co-occurrence of gene/protein names in abstracts).
'''
all_res = []
for row in tqdm(r.itertuples()):    
    # I'm passing the gene ID directly since STRING maps these ID's automatically to the gene's main protein.
    # species 9606 = human.
    url = f"https://string-db.org/api/json/network?identifiers={'%0d'.join([str(row._1), str(row._2)])}&species=9606"
    try:
        response = requests.get(url).json()
        response[0]["Gene 1"] = row._1
        response[0]["Gene 2"] = row._2
        all_res.append(response[0])
    except:
        continue

In [ ]:
'''
My interpretation of the scores (after reading a bit online):
nscore - How close the 2 genes are in the genome (the entire set of organism DNA)
fscore - Evidence of whether the 2 genes fuse together to form a hybrid gene
pscore - Evidence of whether the 2 genes have a common ancestor (homologous)
ascore - Evidence of whether the 2 genes are simultaneously activated or not.
escore - Evidence of whether the 2 genes have been shown to interact through lab experiments.
dscore - Degree of presence in existing datasbases.
tscore - Evidence of reporting in literature.
'''
def return_edge_label(pair: dict):
    pair_scores = {'nscore': pair['nscore'], 'fscore': pair['fscore'], 'pscore': pair['pscore'], 'ascore': pair['ascore'], 
                   'escore': pair['escore'], 'dscore': pair['dscore'], 'tscore': pair['tscore']}
    best_evidence = max(pair_scores.items(), key=lambda x: x[1])
    if best_evidence[0] == 'nscore':
        return "neighborhood_evidence"
    elif best_evidence[0] == 'fscore':
        return "fusion_evidence"
    elif best_evidence[0] == 'pscore':
        return "homologous_evidence"
    elif best_evidence[0] == 'ascore':
        return "co-expression_evidence"
    elif best_evidence[0] == 'escore':
        return "experimental_evidence"
    elif best_evidence[0] == 'dscore':
        return "database_evidence"
    elif best_evidence[0] == 'tscore':
        return "literature_evidence"

In [ ]:
for idx in range(len(all_res)):
    all_res[idx]["relationship"] = return_edge_label(all_res[idx])

pd.DataFrame(all_res)[["Gene 1", "Gene 2", "stringId_A", "stringId_B", "preferredName_A", "preferredName_B", 
                       "relationship"]].to_csv("PPI_subset.csv", index=False)

In [ ]:
# Adding annotation data for new proteins

ppi_subset_genes = set(ppi_subset["Gene 1"].to_list()).union(set(ppi_subset["Gene 1"].to_list()))

def get_protein_annotation(gene_number):
    url = f"https://string-db.org/api/json/get_string_ids?identifiers={gene_number}&species=9606"
    try:
        return requests.get(url).json()[0]["annotation"]
    except:
        return ""

gene_annotations = {}
for row in tqdm(ppi_subset.itertuples(index=False)): 
    gene_1 = row[0]
    if gene_1 not in gene_annotations:
        gene_annotations[gene_1] = get_protein_annotation(gene_1)

    gene_2 = row[1]
    if gene_2 not in gene_annotations:
        gene_annotations[gene_2] = get_protein_annotation(gene_2)

gene_1_annotations = []
gene_2_annotations = []
for row in ppi_subset.itertuples(index=False):
    gene_1_annotations.append(gene_annotations[row[0]])
    gene_2_annotations.append(gene_annotations[row[1]])

ppi_subset["protein_1_desc"] = gene_1_annotations
ppi_subset["protein_2_desc"] = gene_2_annotations

ppi_subset.to_csv("PPI_subset.csv", index=False)

## Collecting Background information

Keywords obtained from https://pubmed.ncbi.nlm.nih.gov/help/#proximity-searching

### Drug info

In [ ]:
ddi_subset = pd.read_csv("DDI_subset.csv")
all_drugs = set(ddi_subset["drug_1_name"].to_list() + ddi_subset["drug_2_name"].to_list())

In [ ]:
def create_drug_info(drug: str):
    '''
    AD - Administration and Dosage
    AE - Adverse Effects
    PK - Pharmacokinetics
    PD - Pharmacology
    CO - Complications
    TU - Therapeutic Use
    DE - Drug Effects
    '''
    drug_query = f"{drug} AND hasabstract AND Humans AND (AD OR AE OR PK OR PD OR CO OR TU OR DE)"

    # Getting relevant papers for given drug
    with Entrez.esearch(db='pubmed', term=drug_query, retmax=20, sort="relevance") as handle:
        paper_list = Entrez.read(handle)["IdList"]
    
    # Getting a parsable record for each paper
    with Entrez.efetch(db='pubmed', rettype='medline', retmode="text", id=paper_list) as handle:
        records = Medline.parse(handle)
        record_list = []
        for rec in records:
            record_list.append(rec)

    # Collecting relevant information
    metadata = []
    all_abstracts_string = ""
    for paper_id, record in zip(paper_list, record_list):
        try:
            metadata.append((drug, paper_id, record["TI"], record["AU"], record["MH"], f"https://pubmed.ncbi.nlm.nih.gov/{paper_id}/"))
            all_abstracts_string = all_abstracts_string + record["AB"] + "\n"
        except:
            continue

    with Path(f"background_information_data/drug_data/{drug}.txt").open("w") as output_file:
        output_file.write(all_abstracts_string)
    
    return metadata

In [ ]:
# Storing drug SMILES data
for row in ddi_subset.itertuples():
    drug1_path = Path(f"background_information_data/drug_data/{row.drug_1_name}_SMILES.txt")
    if drug1_path.exists() is False:
        with drug1_path.open("w") as out_file:
            out_file.write(row.drug_1_SMILES)

    drug2_path = Path(f"background_information_data/drug_data/{row.drug_2_name}_SMILES.txt")
    if drug2_path.exists() is False:
        with drug2_path.open("w") as out_file:
            out_file.write(row.drug_2_SMILES)

In [ ]:
# Gathering all the relevant information 
metadata = []
for drug in tqdm(all_drugs):
    metadata.extend(create_drug_info(drug))

pd.DataFrame(metadata, columns=["drug_name", "pubmed_id", "title", "authors", "mesh_terms", "paper_url"]).to_csv("background_information_data/drug_data/metadata.csv")

### Protein info

In [ ]:
dpi_subset = pd.read_csv("DPI_subset.csv")
ppi_subset = pd.read_csv("PPI_subset.csv")

dpi_proteins = dpi_subset[["protein_name", "protein_desc"]].drop_duplicates().set_index("protein_name")["protein_desc"].to_dict()

ppi_proteins_1 = ppi_subset[["preferredName_A", "protein_1_desc"]].drop_duplicates().set_index("preferredName_A")["protein_1_desc"].to_dict()
ppi_proteins_2 = ppi_subset[["preferredName_B", "protein_2_desc"]].drop_duplicates().set_index("preferredName_B")["protein_2_desc"].to_dict()
ppi_proteins = {**ppi_proteins_1, **ppi_proteins_2}

all_proteins = {**dpi_proteins, **ppi_proteins}

In [ ]:
def create_protein_info(protein: str, protein_desc: str):
    '''
    CH - chemistry
    ME - metabolism
    PH - physiology
    GE - genetics
    '''
    protein_query = f"{protein} AND hasabstract AND Humans AND (CH OR ME OR PH OR GE)"

    # Getting relevant papers for given protein
    with Entrez.esearch(db='pubmed', term=protein_query, retmax=20, sort="relevance") as handle:
        paper_list = Entrez.read(handle)["IdList"]
    
    # Getting a parsable record for each paper
    with Entrez.efetch(db='pubmed', rettype='medline', retmode="text", id=paper_list) as handle:
        records = Medline.parse(handle)
        record_list = []
        for rec in records:
            record_list.append(rec)

    # Collecting relevant information
    metadata = []
    all_abstracts_string = protein_desc + "\n" # Adding the annotation information from STRING
    for paper_id, record in zip(paper_list, record_list):
        try:
            metadata.append((protein, paper_id, record["TI"], record["AU"], record["MH"], f"https://pubmed.ncbi.nlm.nih.gov/{paper_id}/"))
            all_abstracts_string = all_abstracts_string + record["AB"] + "\n"
        except:
            continue

    with Path(f"background_information_data/protein_data/{protein}.txt").open("w") as output_file:
        output_file.write(all_abstracts_string)
    
    return metadata

In [ ]:
metadata = []
for protein_name, protein_desc in tqdm(all_proteins.items()):
    metadata.extend(create_protein_info(protein_name, protein_desc))

pd.DataFrame(metadata, columns=["protein_name", "pubmed_id", "title", "authors", "mesh_terms", "paper_url"]).to_csv("background_information_data/protein_data/metadata.csv")